In [10]:
import time
import datetime
import os
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
import AlphaNet
from AlphaNet.Data import DataLoader
from AlphaNet.Models import Res_LSTM

# read_task
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()
task = task_info["Task"]
task_index = int(input(" please enter the task_id:  "))
Alpha_Name = task.loc[task_index,"Alpha_Name"]
start_date = task.loc[task_index,"start_date"]
end_date = task.loc[task_index,"end_date"]
sequence = task.loc[task_index,"sequence"]
LR = task.loc[task_index,"LR"]
epoch_num = task.loc[task_index,"epoch_num"]
feature_num = task.loc[task_index,"feature_num"]

 please enter the task_id:  217


In [8]:
import shutil
shutil.copytree(r'/home/ShareFolder/lgc/Modules', r'/home/wuwenjun/jupyter_code/Shannon/Modules')

'/home/wuwenjun/jupyter_code/Shannon/Modules'

In [17]:
# Model Loader

loss_function = nn.MSELoss()
model = torch.load("/home/wuwenjun/Alpha_Factor/AlphaNetV4_LSTM/2019-01-01_2019-06-01/" + "best_model.tar")
model_loader = AlphaNet.Model_Loader(model = model,device="cuda:1")
print(model_loader.model)

Res_LSTM(
  (bn1): BatchNorm1d(6180, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(4120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(206, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(103, 103, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (TransformerLayer1): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=103, out_features=103, bias=True)
    )
    (linear1): Linear(in_features=103, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=103, bias=True)
    (norm1): LayerNorm((103,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((103,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (TransformerLayer2): Trans

In [18]:
data_path = "/home/wuwenjun/Data/"
# Test
testloader = DataLoader()
testloader.load_data_from_file(alpha_name = Alpha_Name,start_date = start_date,end_date = end_date,data_path = data_path)
test_loader = testloader.to_torch_DataLoader(shape = [3,sequence,feature_num],shuffle=False)

100%|██████████| 98/98 [01:39<00:00,  1.01s/it]


In [19]:
model_loader.device = "cuda:1"
model_loader.model = model_loader.model.to("cuda:1")

In [20]:
pred = model_loader.pred(test_loader)
# convert to standard daily and back test
final = pd.concat([testloader.target.reset_index(),pred],axis=1)
final.rename(columns={0:Alpha_Name,'ticker': 'symbol'},inplace=True)

In [21]:
model_path= "/home/wuwenjun/Alpha_Factor/AlphaNetV4_LSTM/2020-01-01_2020-06-01/"
final.to_parquet(model_path + "result.par")
convert_to_standard_daily_feature_csv(Alpha_Name, final.drop("target",axis=1), output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun')

# back test

pd.set_option('expand_frame_repr', False)
configs = namespace.load_namespace(r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template')
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")

FT.load_feature_from_file(Alpha_Name, "2019-01-01", end_date, universe='Investable',timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()
FT.get_intersection_ic(feature_name=Alpha_Name, truncate_fold=None, method='spearman',period=('1d', '3d', '5d'))
ic_flag, trading_direction = FT.test_ic(Alpha_Name, verbose=False)
df, df_all = FT.get_ic_summary_by_month(num=6)

if trading_direction == -1:
    negative = True
else:
    negative = False


# save txt
f = open(model_path + 'back_test.txt','a')
print(df,end="\n"*3,file = f)
a = FT.get_top_return(Alpha_Name, negative= False, trade_type='long-only', transmission_rate=0.00025,
                        tax_rate=0.001, verbose=True,bt_price = "vwap",trade_period=6)
print(a[1],end = "\n"*3+"*"*100 ,file = f)
f.close()

Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 341/341 [00:00<00:00, 529.91it/s]



IC table for feature "AlphaNetV4_LSTM" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV4_1d  |  IC_AlphaNetV4_2d  |  IC_AlphaNetV4_3d  |  IC_AlphaNetV4_4d  |  IC_AlphaNetV4_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190102-20190630 |   4.81 4.98 0.97   |   6.21 5.42 1.15   |   6.92 5.91 1.17   |   7.47 6.33 1.18   |   7.96 6.62 1.20   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   2.66 3.59 0.74   |   3.20 3.38 0.95   |   3.59 3.25 1.10   |   3.90 3.26 1.20   |   4.06 3.25 1.25   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101-20200630 |   2.75 5.47 0.50   |   3.34 5.20 0

Getting BT result for date: 20200529...: 100%|██████████████████████████████████████████████████████████████████████| 341/341 [00:05<00:00, 58.51it/s]



AlphaNetV4_LSTM back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     341     |   8.47   |   5.94    |   18.07    |   2.96   |    60.68    | 3.77(20190116-20190131) |  1.28(20190131)   |   48.59    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     343     |  -7.60   |   4.03    |   -14.03   |  -3.92   |    39.20    | 7.92(20190702-20191231) |  0.77(